In [1]:
from pymongo import MongoClient
import pandas as pd
import functions as fun
import os
from dotenv import load_dotenv
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from geopy import distance

# Connecting to the MongoDB

In [2]:
# First of all, we connect with MongoDB

client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("Companies")

In [3]:
# We extract the data from the DB. In this case we are required to extract it according to certain parameters

condition_one = {"description": {"$regex": "design"}}
condition_two = {"tag_list": {"$regex": "design"}}
condition_three = {"name": {"$regex": "design"}}
condition_four = {"category_code": {"$regex": "design"}}
condition_five = {"total_money_raised": {"$regex": "M"}}
condition_six = {"total_money_raised": {"$regex": "B"}}

projection = {"name": 1, "number_of_employees": 1, "offices": 1,"_id":0}
result = list(c.find({
    "$or": [condition_one, condition_two, condition_three, condition_four, condition_five, condition_six],
    "number_of_employees": {"$gte": 100}
}, projection))


df = pd.DataFrame(result)

In [4]:
# Once we got the df, we get all the data that we need from it applying different functions and clean the df:

list_offices = df["offices"]

fun.get_info(list_offices, df,  "Country", "country_code")

fun.get_info(list_offices, df,"Zip_code", "zip_code")

fun.get_info(list_offices, df,"City", "city")

fun.get_info(list_offices, df,"Address", "address1")

fun.get_info(list_offices, df,"Lat", "latitude")

fun.get_info(list_offices, df,"Long", "longitude")

fun.drop_column(df, "offices")

fun.drop_nan(df)

df

,name,number_of_employees,Country,Zip_code,City,Address,Lat,Long
0,Facebook,5299,USA,94025,Menlo Park,1601 Willow Road,37.41605,-122.151801
1,Twitter,1300,USA,94103,San Francisco,1355 Market St.,37.776805,-122.416924
2,eBay,15000,USA,95125,San Jose,2145 Hamilton Avenue,37.295005,-121.930035
3,Cisco,63000,USA,95134,San Jose,170 West Tasman Dr.,37.408802,-121.95377
4,Yahoo!,13600,USA,94089,Sunnyvale,701 First Avenue,37.418531,-122.025485
...,...,...,...,...,...,...,...,...
380,Mashable,100,USA,10010-3646,New York,304 Park Ave South,40.740154,-73.986742
381,SpringSoft,400,USA,95110,San Jose,"2025 Gateway Place, Suite 400",37.37027,-121.924053
382,MentorMate,216,USA,55408,Minneapolis,3036 Hennepin Avenue,44.948786,-93.161038
383,InfoLogix,206,USA,19040,Hatboro,101 East County Line Road,40.192329,-75.106923


- Once we got the df clean, we have a deeper look to the different categories on it in order to have a clearer view of the next steps that we need to take and specially focusing on the different cities where the companies are located.

In [5]:
fun.value_counts(df, "City_count", "City")

In [6]:
# We reduce the number of cities keeping those with at least 5 companies.

df = fun.keep_value(df, "City_count", 5.0)

In [7]:
df["City"].value_counts()

San Francisco    50
New York         30
San Jose         16
Mountain View    16
Palo Alto        11
San Mateo        11
Sunnyvale         9
Seattle           8
London            8
Redwood City      7
Chicago           7
Cambridge         6
null              6
Paris             6
Austin            5
Waltham           5
Los Angeles       5
Name: City, dtype: int64

# Trying to keep different cities in different areas and different locations, we are going to focus on the cities of San Francisco , Seattle, Chicago , London and Cambridge (USA).

In [8]:
San_Francisco = 37.7871253,-122.4015358
sub_SF = fun.do_subset(df, "San Francisco")

Seattle = 47.6251419,-122.3268577
sub_SE = fun.do_subset(df, "Seattle")

Chicago = 41.869276, -87.626694
sub_CH = fun.do_subset(df, "Chicago")

London = 51.4621272,-0.1162654
sub_LO = fun.do_subset(df, "London")

Cambridge = 42.3731956,-71.1198561
sub_CA = fun.do_subset(df, "Cambridge")

In [9]:
load_dotenv()
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about. There are many for each of our locations

In [10]:
starbucks_SF = fun.get_results("starbucks", San_Francisco, 10, 3000, token_foursquare)
df_SF = fun.clean_results_first(starbucks_SF, "Coffee")

starbucks_SE = fun.get_results("starbucks", Seattle, 10, 3000, token_foursquare)
df_SE = fun.clean_results_first(starbucks_SE, "Coffee")

starbucks_LO = fun.get_results("starbucks", London, 10, 3000, token_foursquare)
df_LO = fun.clean_results_first_lo(starbucks_LO, "Coffee")

starbucks_CA = fun.get_results("starbucks", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results_first(starbucks_CA, "Coffee")

starbucks_CH = fun.get_results("starbucks", Chicago, 10, 3000, token_foursquare)
df_CH = fun.clean_results_first(starbucks_CH, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about. There are also enough within a short distance from the offices.

In [11]:
kinder_SF = fun.get_results("preschool", San_Francisco, 10, 3000, token_foursquare)
df_SF = fun.clean_results(kinder_SF, df_SF, "Preschool")

kinder_SE = fun.get_results("preschool", Seattle, 10, 3000, token_foursquare)
df_SE = fun.clean_results(kinder_SE, df_SE, "Preschool")

kinder_LO = fun.get_results("Nursery school", London, 5, 3000, token_foursquare)
df_LO = fun.clean_results_lo(kinder_LO, df_LO, "Preschool")

kinder_CA = fun.get_results("preschool", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(kinder_CA, df_CA, "Preschool")

kinder_CH = fun.get_results("preschool", Chicago, 10, 3000, token_foursquare)
df_CH = fun.clean_results(kinder_CH, df_CH, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about

In [12]:
club_SF = fun.get_results("night club", San_Francisco, 10, 3000, token_foursquare)
df_SF = fun.clean_results(club_SF, df_SF, "Club")

club_SE = fun.get_results("night club", Seattle, 10, 3000, token_foursquare)
df_SE = fun.clean_results(club_SE, df_SE, "Club")

club_LO = fun.get_results("night club", London, 5, 3000, token_foursquare)
df_LO = fun.clean_results_lo(club_LO, df_LO, "Club")

club_CA = fun.get_results("night club", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(club_CA, df_CA, "Club")

club_CH = fun.get_results("night club", Chicago, 10, 3000, token_foursquare)
df_CH = fun.clean_results(club_CH, df_CH, "Club")

- We look for vegan restaurants around the office. There are less and some that are not explicitely "vegans".

In [13]:
vegan_SF = fun.get_results("vegan", San_Francisco, 10, 3000, token_foursquare)
df_SF = fun.clean_results(vegan_SF, df_SF, "Vegan restaurant")

vegan_SE = fun.get_results("vegan", Seattle, 10, 3000, token_foursquare)
df_SE = fun.clean_results(vegan_SE, df_SE, "Vegan restaurant")

vegan_LO = fun.get_results("vegan", London, 10, 3000, token_foursquare)
df_LO = fun.clean_results_lo(vegan_LO, df_LO, "Vegan restaurant")

vegan_CA = fun.get_results("vegan", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(vegan_CA, df_CA, "Vegan restaurant")

vegan_CH = fun.get_results("vegan", Chicago, 10, 3000, token_foursquare)
df_CH = fun.clean_results(vegan_CH, df_CH, "Vegan restaurant")

- We look for basketball arenas. Some of them are from major NBA teams, so we look for the actual name of its stadiums. However in Seattle, there is no local team at the moment (we pray for the Supersonics return soon). In London there is an arena, with a local team, however, is not within a "short" distance from the office.

In [14]:
Basketball_SF = fun.get_results("Chase center", San_Francisco, 1, 5000, token_foursquare)
df_SF = fun.clean_results(Basketball_SF, df_SF, "Basketball arena")

Basketball_SE = fun.get_results("Climate Pledge Arena", Seattle, 1, 5000, token_foursquare)
df_SE = fun.clean_results(Basketball_SE, df_SE, "Basketball arena")

Basketball_LO = fun.get_results("Basketball arena", London, 1, 5000, token_foursquare)

Basketball_CA = fun.get_results("TD Garden", Cambridge, 1, 5000, token_foursquare)
df_CA = fun.clean_results(Basketball_CA, df_CA, "Basketball arena")

Basketball_CH = fun.get_results("United center", Chicago, 1, 5000, token_foursquare)
df_CH = fun.clean_results(Basketball_CH, df_CH, "Basketball arena")


- We look for dog hairdressers around the office; we did not find any result for the cities of London and Cambridge.

In [15]:
pet_SF = fun.get_results("pet grooming", San_Francisco, 5, 3000, token_foursquare)
df_SF = fun.clean_results(pet_SF, df_SF, "Pet hairdresser")

pet_SE = fun.get_results("pet grooming", Seattle, 5, 3000, token_foursquare)
df_SE = fun.clean_results(pet_SE, df_SE, "Pet hairdresser")

pet_LO = fun.get_results("dog grooming", London, 5, 3000, token_foursquare)
df_LO = fun.clean_results_lo(pet_LO, df_LO, "Pet hairdresser")

pet_CA = fun.get_results("dog grooming", Cambridge, 5, 3000, token_foursquare)
df_CA = fun.clean_results_lo(pet_CA, df_CA, "Pet hairdresser")

pet_CH = fun.get_results("pet grooming", Chicago, 5, 3000, token_foursquare)
df_CH = fun.clean_results(pet_CH, df_CH, "Pet hairdresser")

- We look for the closest Airport to office. In all the cities except Seattle, we look for two airports.

In [16]:
Airports_SF = fun.get_results_airport("Airport", San_Francisco, 2, 30000, token_foursquare)
df_SF = fun.clean_results_lo(Airports_SF, df_SF, "Airport")

Airports_SE = fun.get_results_airport("Seattle tacoma airport", Seattle, 1, 30000, token_foursquare)
df_SE = fun.clean_results_lo(Airports_SE, df_SE, "Airport")

Airports_LO = fun.get_results_airport("Airport", London, 2, 30000, token_foursquare)
df_LO = fun.clean_results_lo(Airports_LO, df_LO, "Airport")

Airports_CA = fun.get_results_airport("Airport", Cambridge, 2, 30000, token_foursquare)
df_CA = fun.clean_results_lo(Airports_CA, df_CA, "Airport")

Airports_CH = fun.get_results_airport("Airport", Chicago, 2, 30000, token_foursquare)
df_CH = fun.clean_results(Airports_CH, df_CH, "Airport")

- We add the data of the new office that we have selected. This is a random location close to the places that we have looked for and the companies from the MongoDB

In [17]:
df_SF = fun.add_company(df_SF, "662 Mission St, San Francisco, CA 94105", "Soma", 37.7871253, -122.4015358)

df_SE = fun.add_company(df_SE, "South Lake Union, Seattle, Washington 98109", "South Lake Union", 47.624730, -122.335911)

df_LO = fun.add_company(df_LO, "1-2 More London Pl, London", "Southwark", 51.505024, -0.080416)

df_CA = fun.add_company(df_CA, "5, John F. Kennedy St, Cambridge, MA 02138", "West Cambridge", 42.3731956,-71.1198561)

df_CH = fun.add_company(df_CH, "118 S Holden Ct, Chicago, IL 60605", "Dearborn Park", 41.869276, -87.626694)

- We create a new row with the distance from the office to the diferent places (we do not include the distance to the other companies here, as we consider that is enough sharing the city)

In [18]:
df_SF["Distance"] = df_SF.apply(fun.calc_distance_SF, axis = 1).round(2)

df_SE["Distance"] = df_SE.apply(fun.calc_distance_SE, axis = 1).round(2)

df_LO["Distance"] = df_LO.apply(fun.calc_distance_LO, axis = 1).round(2)

df_CA["Distance"] = df_CA.apply(fun.calc_distance_CA, axis = 1).round(2)

df_CH["Distance"] = df_CH.apply(fun.calc_distance_CH, axis = 1).round(2)

In [19]:
df_SF

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"662 Mission St, San Francisco, CA 94105",Soma,37.787125,-122.401536,NaN,0.00
1,Starbucks,"299 Fremont St (Folsom St), San Francisco, CA ...",Chinatown,37.788371,-122.393555,Coffee,716.46
2,Starbucks,"789 Mission St (at 4th St), San Francisco, CA ...",Civic Center,37.784470,-122.404008,Coffee,366.45
3,Starbucks,"7 Drumm St (California), San Francisco, CA 94111",Chinatown,37.793818,-122.396678,Coffee,857.26
4,Starbucks,"865 Market St (5th street), San Francisco, CA ...",Civic Center,37.784144,-122.407429,Coffee,615.61
5,Starbucks,"780 Market St (at 4th St), San Francisco, CA 9...",Civic Center,37.786145,-122.405612,Coffee,375.18
6,Starbucks,"462 Powell St (at Sutter St), San Francisco, C...",Chinatown,37.789098,-122.408438,Coffee,646.20
7,Starbucks,"390 Stockton St (at Sutter St), San Francisco,...",Chinatown,37.789293,-122.406729,Coffee,516.86
8,Starbucks,"74 New Montgomery St (at Jessie St), San Franc...",Financial District,37.787664,-122.401113,Coffee,70.44
9,Starbucks,"565 Clay St (at Montgomery St), San Francisco,...",Chinatown,37.794601,-122.402669,Coffee,835.73


- Then we create the base maps where we will add the different layers:

In [20]:
SF_map = Map(location = San_Francisco, zoom_start = 12)
SE_map = Map(location = Seattle, zoom_start = 12)
LO_map = Map(location = London, zoom_start = 12)
CA_map = Map(location = Cambridge, zoom_start = 12)
CH_map = Map(location = Chicago, zoom_start = 12)

- We create select the Icons and select the categories that we want to be displayed on the map

In [21]:
fun.mapping_results(df_SF, SF_map)
fun.mapping_results(df_SE, SE_map)
fun.mapping_results(df_LO, LO_map)
fun.mapping_results(df_CA, CA_map)
fun.mapping_results(df_CH, CH_map)

- Then we add the companies from the MongoDB to the map

In [22]:
fun.mapping_companies(sub_SF, SF_map)
fun.mapping_companies(sub_SE, SE_map)
fun.mapping_companies(sub_LO, LO_map)
fun.mapping_companies(sub_CA, CA_map)
fun.mapping_companies(sub_CH, CH_map)

- And here we got the maps!!

In [23]:
SF_map

In [24]:
SE_map

In [25]:
LO_map

In [26]:
CA_map

In [27]:
CH_map

# Weighting the results

- First of all we delete the first element of each list, that correspond to our company details.

In [28]:
df_SF = df_SF.iloc[1:]
df_CA = df_CA.iloc[1:]
df_CH = df_CH.iloc[1:]
df_SE = df_SE.iloc[1:]
df_LO = df_LO.iloc[1:]

- In order to to get the final result for each city, we just apply a simple math formulation to the grouped data for each city. The one with the lowest score will be the winner.


# San Francisco

In [29]:
index = pd.Index(['Airport','Basketball arena','Club','Coffee','Pet hairdresser',"Preschool","Vegan restaurant"], name = "Category")
SF_weight = pd.DataFrame({'Weight factor':[0.1, 0.2, 0.05, 0.1, 0.2, 0.05, 0.3]}, index=index)
SF_weight.insert(1, 'Radius', [30000, 5000, 3000, 3000, 3000, 3000,3000])

SF_weight = fun.get_weight(SF_weight, df_SF)

print(fun.final_result(SF_weight, "San Francisco"))
print(SF_weight)

The final result for San Francisco is 0.37
                  Weight factor  Radius       Mean       Max       Min  \
Airport                    0.10   30000  18352.690  18605.79  18099.59   
Basketball arena           0.20    5000   2470.210   2470.21   2470.21   
Club                       0.05    3000   1816.876   2613.47    945.47   
Coffee                     0.10    3000    544.230    857.26     70.44   
Pet hairdresser            0.20    3000    698.642   1134.74    398.67   
Preschool                  0.05    3000   1474.100   2907.50    521.78   
Vegan restaurant           0.30    3000    930.075   1593.07    224.02   

                  Final Result  
Airport                   0.06  
Basketball arena          0.10  
Club                      0.03  
Coffee                    0.02  
Pet hairdresser           0.05  
Preschool                 0.02  
Vegan restaurant          0.09  


# Cambridge

In [30]:
index = pd.Index(['Airport','Basketball arena','Club','Coffee','Pet hairdresser',"Preschool","Vegan restaurant"], name = "Category")
CA_weight = pd.DataFrame({'Weight factor':[0.1, 0.2, 0.05, 0.1, 0.2, 0.05, 0.3]}, index=index)
CA_weight.insert(1, 'Radius', [30000, 5000, 3000, 3000, 3000, 3000,3000])

CA_weight = fun.get_weight(CA_weight, df_CA)
fun.final_result(CA_weight, "Cambridge")

'The final result for Cambridge is 0.65'

# Chicago

In [31]:
index = pd.Index(['Airport','Basketball arena','Club','Coffee','Pet hairdresser',"Preschool","Vegan restaurant"], name = "Category")
CH_weight = pd.DataFrame({'Weight factor':[0.1, 0.2, 0.05, 0.1, 0.2, 0.05, 0.3]}, index=index)
CH_weight.insert(1, 'Radius', [30000, 5000, 3000, 3000, 3000, 3000,3000])
CH_weight = fun.get_weight(CH_weight, df_CH)

fun.final_result(CH_weight, "Chicago")

'The final result for Chicago is 0.56'

# Seattle

In [32]:
index = pd.Index(['Airport','Basketball arena','Club','Coffee','Pet hairdresser',"Preschool","Vegan restaurant"], name = "Category")
SE_weight = pd.DataFrame({'Weight factor':[0.1, 0.2, 0.05, 0.1, 0.2, 0.05, 0.3]}, index=index)
SE_weight.insert(1, 'Radius', [30000, 5000, 3000, 3000, 3000, 3000,3000])

SE_weight = fun.get_weight(SE_weight, df_SE)
fun.final_result(SE_weight, "Seattle")

'The final result for Seattle is 0.52'

# London

In [33]:
index = pd.Index(['Airport','Basketball arena','Club','Coffee','Pet hairdresser',"Preschool","Vegan restaurant"], name = "Category")
LO_weight = pd.DataFrame({'Weight factor':[0.1, 0.2, 0.05, 0.1, 0.2, 0.05, 0.3]}, index=index)
LO_weight.insert(1, 'Radius', [30000, 5000, 3000, 3000, 3000, 3000,3000])

LO_weight = fun.get_weight(LO_weight, df_LO)
fun.final_result(LO_weight, "London")

'The final result for London is 1.28'

# The city selected is San Francisco